In [4]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from flask_socketio import SocketIO,emit
# Load the saved random forest model for texture: wood
# loaded_model = joblib.load('wood_RF_model.joblib')

#collison flag
collisionOnflag = False

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return jsonify({'message': 'Welcome to the WebSocket server!'})

@socketio.on('connect')
def test_connect():
    socketio.emit('after connect', {'data':'Let us learn Web Socket in Flask'})

@app.route('/collisonRegister', methods=['POST'])
def collisonRegister():
    global collisionOnflag
    try:
        collisionStatus = request.json.get('CollisionStatus')  
        textureName = request.json.get('TextureName')
        
        if collisionStatus is None:
            return jsonify({'Status': '400', 'Error': 'No data received'})
        
        elif collisionStatus not in ['ON', 'OFF']:
            return jsonify({'Status': '400', 'Error': 'Invalid CollisionStatus'})
        
        elif collisionStatus == "ON" and collisionOnflag == False:
            if textureName is None or textureName.strip() == '':
                return jsonify({'Status': '400', 'Error': 'Texture name is empty or null'})
            else:
                #start to listen input for the cane velocity
                #for each velocity input from the cane, use the current texture in use to call the prediction model
                #return the predicted DFT321 waveform value
                collisionOnflag = True
                return jsonify({'Status': '200', 'Message': 'CollisionStatus is ON'})
        elif collisionStatus == "OFF" and collisionOnflag == True:  
            #collisionStatus is 'OFF'
            #stop listening to the cane input
            collisionOnflag = False
            return jsonify({'Status': '200', 'Message': 'CollisionStatus is OFF'})
        else:
            return jsonify({'Status': '400', 'Message': 'CollisionStatus is ALREADY ON'})
    
    except Exception as e:
        return jsonify({'Status': '500', 'Error': str(e)} )
    
@app.route('/predict', methods=['POST'])
def predict():
    global collisionOnflag
    if collisionOnflag == True:
        try:
                data = request.get_json()
                # Convert JSON data to a Python dictionary
                vector_data = {
                    'VELO-X': data.get('x'),
                    'VELO-Y': data.get('y'),
                    'VELO-Z': data.get('z')
                }

                x_input_df = pd.DataFrame([vector_data])
                # y_pred = loaded_model.predict(x_input_df)

                # return jsonify({'Status': '200', 'Message': str(y_pred)})
        except Exception as e:
                return jsonify({'Status': '500', 'Error': str(e)}) 
    else:
        return jsonify({'Status': '400', 'Error': 'Collision flag is not set to true'}) 

# Endpoint to receive a manual message and forward it to ESP32 via WebSocket
@app.route('/manual_message', methods=['POST'])
def manual_message():
    try:
        data = request.get_json()
        message = data.get('message')

        if message:
            # Forward the manual message to ESP32 via WebSocket
            socketio.emit('message_to_esp32', {'message': message}, broadcast=True)
            return jsonify({'status': 'success', 'message': 'Manual message forwarded to ESP32'})
        else:
            return jsonify({'status': 'error', 'message': 'Invalid or missing message in the request'})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})



if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.4.57:8080
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.4.96 - - [13/Jan/2024 19:52:47] "GET / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/2024 19:53:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/2024 19:53:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/2024 19:53:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/2024 19:53:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.4.96 - - [13/Jan/2024 19:53:11] "GET / HTTP/1.1" 400 -
INFO:werkzeug:192.168.4.96 - - [13/Jan/2024 20:54:24] "GET / HTTP/1.1" 400 -
INFO:werkzeug:192.168.4.96 - - [13/Jan/2024 20:55:22] "GET / HTTP/1.1" 400 -
INFO:werkzeug:192.168.4.96 - - [13/Jan/2024 20:56:06] "GET / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [13/Jan/20